Assignment Lab 7b

Use the interactive1.py code in numlabs/lab7 for this question.


If your experience with the Coriolis force is minimal, you can chose the
“small” option. See the doc string for reasonable parameters.


The code solves the high water level in the center (similar to rain.py)
in two-dimensions with periodic boundary conditions on a flat bottom.
The depth is set in the functions find_depth*. You will use the C-grid and edit the
find_depth3 function as described below.


(a) Choose an interesting but smooth topography (remembering that the domain is periodic in both space dimensions). Implement it in find_depth3 correctly given the grid-C staggering.


(b) Run your new code. Discuss any other changes you make to the code. You may want to change what and when it plots.


(c) Explain the differences that the bottom topography makes.